In [14]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

## This sets the plot style
## to have a grid on a white background
set_style("darkgrid")

In [15]:
X_orig = np.load('../../data/processed_data/specgram_db_pad_trunc.npy')

X_specaug1 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask1.npy')
X_tsh1 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift1.npy')
X_randn1 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise1.npy')
X_tstr1 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch1.npy')

X_specaug2 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask2.npy')
X_tsh2 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift2.npy')
X_randn2 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise2.npy')
X_tstr2 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch2.npy')

X_specaug3 = np.load('../../data/processed_data/augment_data/spec_pt_ftmask3.npy')
X_tsh3 = np.load('../../data/processed_data/augment_data/spec_pt_timeshift3.npy')
X_randn3 = np.load('../../data/processed_data/augment_data/spec_pt_randnoise3.npy')
X_tstr3 = np.load('../../data/processed_data/augment_data/spec_pt_timestretch3.npy')

print(X_orig.shape)

(440, 128, 67)


In [16]:
#choose what to add to the orignal data
X = np.concatenate((X_orig,X_specaug3,X_tstr3), axis=0)
print(X.shape)

df = pd.read_csv("../../data/processed_data/metadata.csv")
category_to_number = {'brushing': 0, 'food': 1, 'isolation': 2}

# Create a new column with numerical values based on the mapping
df['numerical_situation'] = df['situation'].map(category_to_number)

y_orig = df['numerical_situation'].values
y = np.concatenate((y_orig, y_orig, y_orig), axis=0)
print(y.shape)

(1320, 128, 67)
(1320,)


In [17]:
# NEW to CONV nets
## We need to reshape our data to be the correct dimensions for
## the net
## observations, grid, colorvalues

##so we just need to add the colorvalues, 1 here represents single valued
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
print(X.shape)

(1320, 128, 67, 1)


In [18]:
def plottv(history_dict,epochs):
    plt.figure(figsize = (8,6))

    plt.scatter(range(1,epochs+1), history_dict['accuracy'], label = "Training Accuracy")
    plt.scatter(range(1,epochs+1), history_dict['val_accuracy'], label = "Validation Set Accuracy")

    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Accuracy", fontsize=12)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    plt.legend(fontsize=12)

    plt.show()


    plt.figure(figsize = (8,6))

    plt.scatter(range(1,epochs+1), history_dict['loss'], label = "Training Loss")
    plt.scatter(range(1,epochs+1), history_dict['val_loss'], label = "Validation Set Loss")

    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Loss Function Value", fontsize=12)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

    plt.legend(fontsize=12)

    plt.show()

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                       test_size=.2,
                                                       shuffle=True,
                                                       random_state=100,
                                                       stratify=y)

In [20]:
## Importing the things
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold

In [22]:
## Cross Validation folds
kfold = StratifiedKFold(3,
                        shuffle=True,
                        random_state=846)
epochs=100

for train_index, test_index in kfold.split(X_train,y_train):
    
    X_tt, X_val = X[train_index], X[test_index]
    y_tt, y_val = y[train_index], y[test_index]
    
    model = models.Sequential()
    model.add( layers.Conv2D(32, (6,4), activation='relu', input_shape=(X.shape[1],X.shape[2],1,) ) )
    model.add( layers.MaxPooling2D( (3,2), strides = 2 ) )
    model.add( layers.Conv2D(64, (6,4), activation='relu'))
    model.add( layers.MaxPooling2D( (3,2), strides = 2) )
    model.add( layers.Conv2D(96, (6,4), activation='relu'))
    model.add( layers.MaxPooling2D( (3,2), strides = 2) )
    
    
    model.add( layers.Flatten() )
    model.add(layers.Dense(45, activation='relu'))

    ## Finally an output layer
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_tt, 
                          to_categorical(y_tt), 
                          epochs=epochs, 
                          batch_size=int(len(y_tt)),
                          validation_data=(X_val,to_categorical(y_val)))
    history_dict = history.history
    plottv(history_dict,epochs)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.3476 - accuracy: 0.4759 - val_loss: 42.8012 - val_accuracy: 0.2699
Epoch 2/100
1/1 [==============================] - 2s 2s/step - loss: 40.7649 - accuracy: 0.2784 - val_loss: 25.2084 - val_accuracy: 0.2188
Epoch 3/100


KeyboardInterrupt: 

In [8]:
epochs = 100 ##earlier stop, observed from Cross validation

model = models.Sequential()
model.add( layers.Conv2D(32, (6,4), activation='relu', input_shape=(X.shape[1],X.shape[2],1,) ) )
model.add( layers.MaxPooling2D( (3,2), strides = 2 ) )
model.add( layers.Conv2D(64, (6,4), activation='relu'))
model.add( layers.MaxPooling2D( (3,2), strides = 2) )
model.add( layers.Conv2D(96, (6,4), activation='relu'))
model.add( layers.MaxPooling2D( (3,2), strides = 2) )
    
    
model.add( layers.Flatten() )
model.add(layers.Dense(45, activation='relu'))

## Finally an output layer
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


history = model.fit(X_train, 
                    to_categorical(y_train), 
                    epochs=epochs, 
                    batch_size=len(y_train)
                    )
#history_dict = history.history
#plottv(history_dict,epochs)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 4.8149 - accuracy: 0.2888
Epoch 2/100
1/1 [==============================] - 3s 3s/step - loss: 17.6588 - accuracy: 0.5019
Epoch 3/100
1/1 [==============================] - 3s 3s/step - loss: 5.1793 - accuracy: 0.5019
Epoch 4/100
1/1 [==============================] - 3s 3s/step - loss: 1.8981 - accuracy: 0.2093
Epoch 5/100
1/1 [==============================] - 3s 3s/step - loss: 1.2971 - accuracy: 0.2888
Epoch 6/100
1/1 [==============================] - 3s 3s/step - loss: 1.2404 - accuracy: 0.2879
Epoch 7/100
1/1 [==============================] - 3s 3s/step - loss: 1.1134 - accuracy: 0.2898
Epoch 8/100
1/1 [==============================] - 3s 3s/step - loss: 1.0488 - accuracy: 0.5019
Epoch 9/100
1/1 [==============================] - 3s 3s/step - loss: 1.1009 - accuracy: 0.4915
Epoch 10/100
1/1 [==============================] - 3s 3s/step - loss: 1.0406 - accuracy: 0.5028
Epoch 11/100
1/1 [====================

1/1 [==============================] - 3s 3s/step - loss: 0.4281 - accuracy: 0.8438
Epoch 86/100
1/1 [==============================] - 3s 3s/step - loss: 0.4189 - accuracy: 0.8475
Epoch 87/100
1/1 [==============================] - 3s 3s/step - loss: 0.4113 - accuracy: 0.8532
Epoch 88/100
1/1 [==============================] - 3s 3s/step - loss: 0.4057 - accuracy: 0.8542
Epoch 89/100
1/1 [==============================] - 3s 3s/step - loss: 0.3979 - accuracy: 0.8523
Epoch 90/100
1/1 [==============================] - 3s 3s/step - loss: 0.3900 - accuracy: 0.8580
Epoch 91/100
1/1 [==============================] - 3s 3s/step - loss: 0.3824 - accuracy: 0.8636
Epoch 92/100
1/1 [==============================] - 3s 3s/step - loss: 0.3761 - accuracy: 0.8646
Epoch 93/100
1/1 [==============================] - 3s 3s/step - loss: 0.3687 - accuracy: 0.8598
Epoch 94/100
1/1 [==============================] - 3s 3s/step - loss: 0.3597 - accuracy: 0.8636
Epoch 95/100
1/1 [=========================

In [9]:
model.evaluate(X_test,to_categorical(y_test)) ##If want to use this predict new data, use model.predict() after loading the model



9/9 [==============================] - 0s 24ms/step - loss: 0.4423 - accuracy: 0.8144


[0.44233277440071106, 0.814393937587738]

In [10]:
predictions = model.predict(X_test)

9/9 [==============================] - 0s 24ms/step


In [11]:
predicted_categ = np.argmax(predictions, axis=1)

# 'predicted_labels' now contains the predicted class indices for each sample
print(predicted_categ)

[2 0 2 2 2 0 0 0 0 2 2 2 2 2 2 1 0 0 2 2 2 0 2 1 0 2 1 0 0 0 1 2 0 2 2 2 2
 2 2 1 1 0 0 0 1 2 0 2 2 0 2 2 1 2 2 1 2 2 0 0 2 2 0 0 1 1 2 2 0 0 0 0 0 0
 2 1 2 2 0 2 0 2 2 2 1 2 2 2 2 1 2 2 1 2 2 0 0 2 2 2 1 0 2 2 1 2 2 2 2 2 2
 2 2 2 0 0 1 1 2 1 0 1 2 2 0 1 1 2 0 1 0 2 0 2 1 2 2 0 2 2 0 0 1 0 2 1 1 2
 0 1 0 1 2 0 2 1 2 2 1 1 1 1 2 2 1 0 2 1 2 2 2 0 2 2 2 1 1 2 2 1 2 2 2 0 2
 0 0 1 1 2 2 0 2 2 2 0 2 2 2 0 2 0 2 2 1 0 2 2 0 1 0 2 2 2 2 0 1 2 1 0 2 2
 0 0 1 2 0 2 1 2 2 2 1 2 0 2 2 2 1 1 0 1 0 2 1 0 0 0 1 0 2 0 1 2 1 0 2 1 2
 1 2 1 1 1]


In [12]:
predicted_categ-y_test

array([ 0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0, -2,  0,  1,  0,  0,  0, -2,  0, -1,  0,  0,  0,  0,
        0,  2,  0,  0,  0, -1,  0, -1,  0,  0,  0,  0,  0,  0,  2,  0,  0,
        0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       -2,  0,  0,  0, -2, -2,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -2, -2,  0,  0,  0,  0,  0,
        0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  1,  0,
        0, -1,  0,  0,  0,  0,  1,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  2,
        0,  0,  2,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0, -2,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1,  0,  0,
        1, -2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,
        0,  0,  0,  0,  0

In [13]:
model.save("/Users/jinjingyi/meow/meow-by-meow/data/trained_models/CNN/CNN_dataaug_with_ftmask_randn.keras")